In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape, Input, Flatten
from sklearn.model_selection import train_test_split

In [2]:
import time, datetime
from fyers_apiv3 import fyersModel
client_id = "ZHQ4IJL7TI-100"
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA
from indicators.candle import OHLC

In [5]:
date_yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
date_100_p = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime("%Y-%m-%d")
date_yesterday, date_100_p

('2024-11-25', '2024-08-18')

In [6]:
response = fyers.history(data={
                                "symbol": "NSE:ZEEL-EQ",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_100_p,
                                "range_to": date_yesterday,
                                "cont_flag": "1"
                                })

len(response['candles'])

25185

In [7]:
date_temp = []
final_candles_list = []
for i in response['candles']:
    if i[0] not in date_temp:
        final_candles_list.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list)

25185

In [8]:
candles = np.array(final_candles_list)
candles.shape

(25185, 6)

In [9]:
sync = SyncInd(
    SMA(5),
    # Alligator(show_jaw=False, show_teeth=False)
    KAMA(highlight=True)
)

for c in candles:
    sync.append(c)
sync.data().shape

(25185, 9)

In [10]:
def sma_slope_positive(ind, data, candle_check=3, sma_ind=6):
    last_c = data[ind][sma_ind]
    for i in range(1, candle_check+1):
        if last_c < data[ind+i][sma_ind]:
            last_c = data[ind+i][sma_ind]
        else:
            return False
    return True

In [11]:
X_train, y_train = [], []
X_test, y_test = [], []

date_wise_data = {}

temp = []
for ind, i in enumerate(sync.data()):
    k = i.tolist()
    temp.append(k[:5] + k[6:]) # deleting date and volume from the data
current_data = np.array(temp, dtype=np.float32)


for i in current_data:
    if not datetime.datetime.fromtimestamp(int(i[0])).strftime("%D") in date_wise_data.keys():
        date_wise_data[datetime.datetime.fromtimestamp(int(i[0])).strftime("%D")] = [i ]
    else:
        date_wise_data[datetime.datetime.fromtimestamp(int(i[0])).strftime("%D")].append(i)


last_candles_count = 8
next_candles_count = 5

# training data for the day 2 to -2
for i in list(date_wise_data.keys())[1:-1]:
    for j in range(len(date_wise_data[i])):
        ind = last_candles_count + j
        if ind+next_candles_count < len(date_wise_data[i]):
            if sma_slope_positive(ind, date_wise_data[i], sma_ind=5, candle_check=next_candles_count):
                temp = []
                first_candle = date_wise_data[i][ind-last_candles_count]
                for k in range(1, last_candles_count+1):
                    temp.append(date_wise_data[i][ind-last_candles_count+k] - first_candle)
                X_train.append(temp)
                temp = []
                for l in range(1, next_candles_count+1):
                    temp.append(date_wise_data[i][ind+l] - first_candle)
                y_train.append(temp)

# testing data for last day in the data
for i in list(date_wise_data.keys())[-2:-1]:
    for j in range(len(date_wise_data[i])):
        ind = last_candles_count + j
        if ind+next_candles_count < len(date_wise_data[i]):
            if sma_slope_positive(ind, date_wise_data[i], sma_ind=5, candle_check=next_candles_count):
                temp = []
                first_candle = date_wise_data[i][ind-last_candles_count]
                for k in range(1, last_candles_count+1):
                    temp.append(date_wise_data[i][ind-last_candles_count+k] - first_candle)
                X_test.append(temp)
                temp = []
                for l in range(1, next_candles_count+1):
                    temp.append(date_wise_data[i][ind+l] - first_candle)
                y_test.append(temp)

X_train = np.array(X_train)[:, :, 1:]
y_train = np.array(y_train)[:, :, 5]
X_test = np.array(X_test)[:, :, 1:]
y_test = np.array(y_test)[:, :, 5]

In [12]:
X_test_seq, y_test_seq = [], []
for i in list(date_wise_data.keys())[-2:-1]:
    for j in range(len(date_wise_data[i])):
        ind = last_candles_count + j
        if ind+next_candles_count < len(date_wise_data[i]):
            if 1:
                temp = []
                first_candle = date_wise_data[i][ind-last_candles_count]
                for k in range(1, last_candles_count+1):
                    temp.append(date_wise_data[i][ind-last_candles_count+k] - first_candle)
                X_test_seq.append(temp)
                temp = []
                for l in range(1, next_candles_count+1):
                    temp.append(date_wise_data[i][ind+l] - first_candle)
                y_test_seq.append(temp)


X_test_seq = np.array(X_test_seq)[:, :, 1:]
y_test_seq = np.array(y_test_seq)[:, :, 5]

In [13]:
z = -1

In [14]:
z += 1
plt.plot(y_train[z, :])

In [15]:
# X = tf.constant(X, dtype=tf.float32)
# y = tf.constant(y, dtype=tf.float32)

# # X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), test_size=0.05, random_state=42)
# # X_train, X_test, y_train, y_test = X[:int(X.shape[0]*0.95)], X[:int(X.shape[0]*0.05)], y[:int(y.shape[0]*0.95)], y[:int(y.shape[0]*0.05)]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2516, 8, 7), (2516, 5), (34, 8, 7), (34, 5))

In [16]:
model = Sequential()
model.add(Input(shape=X_train.shape[1:]))
model.add(LSTM(200, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(next_candles_count))
model.add(Reshape(y_train.shape[1:]))
model.compile(optimizer='adam', loss='mse')

In [ ]:
%%timeit
with tf.device("/CPU:0"):
    history = model.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2)

Epoch 1/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.5187 - val_loss: 0.0324
Epoch 2/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.1720 - val_loss: 0.0253
Epoch 3/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.2285 - val_loss: 0.0190
Epoch 4/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.1063 - val_loss: 0.0149
Epoch 5/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1352 - val_loss: 0.0147
Epoch 6/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0795 - val_loss: 0.0147
Epoch 7/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.1193 - val_loss: 0.0125
Epoch 8/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0778 - val_loss: 0.0134
Epoch 9/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1097 - val_loss: 0.0185
Epoch 10/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0971 - val_loss: 0.0121
Epoch 11/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0801 - val_loss: 0.0120
Epoch 12/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step

In [ ]:
%%timeit
with tf.device("/GPU:0"):
    history = model.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2)

In [ ]:
y_pred = model.predict(X_test_seq)
# y_pred

In [ ]:
# Example for the first feature (e.g., Close price)
plt.plot(y_test[:50, 0], label='C1 Open', color="pink")
plt.plot(y_test[:50, 1], label='C2 Open', color="lightblue")
plt.plot(y_test[:50, 2], label='C3 Open', color="lightgreen")
plt.plot(y_pred[:50, 0], label='C1 P_Open', color="darkred")
plt.plot(y_pred[:50, 1], label='C2 P_Open', color="darkblue")
plt.plot(y_pred[:50, 2], label='C3 P_Open', color="green")
plt.legend()
plt.show()

In [ ]:
plt.plot(y_test_seq[-50:, 0], label='C1 Open', color="pink")
# plt.plot(y[0:5, 1], label='C2 Open', color="blue")
# plt.plot(y[0:5, 2], label='C3 Open', color="green")
plt.plot(y_pred[-50:, 0], label='C1 P_Open', color="red")
# plt.plot(y_pred[0:5, 1], label='C2 P_Open', color="darkblue")
# plt.plot(y_pred[0:5, 2], label='C3 P_Open', color="darkgreen")
plt.legend()
plt.show()

In [ ]:
plt.plot(y_test_seq[-50:, 1], label='C2 Open', color="lightblue")
# plt.plot(y[0:5, 2], label='C3 Open', color="green")
# plt.plot(y_pred[0:5, 0], label='C1 P_Open', color="darkred")
plt.plot(y_pred[-50:, 1], label='C2 P_Open', color="darkblue")
# plt.plot(y_pred[0:5, 2], label='C3 P_Open', color="darkgreen")
plt.legend()
plt.show()

In [ ]:
plt.plot(y_test_seq[-50:, 2], label='C3 Open', color="lightgreen")
# plt.plot(y_pred[0:5, 0], label='C1 P_Open', color="darkred")
# plt.plot(y_pred[0:5, 1], label='C2 P_Open', color="darkblue")
plt.plot(y_pred[-50:, 2], label='C3 P_Open', color="green")
plt.legend()
plt.show()

In [ ]:
model.save("26NOV_2.keras")

In [ ]:
date_today = datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
response_rt = fyers.history(data={
                                "symbol": "NSE:ZEEL-EQ",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_today,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })

date_temp = []
final_candles_list_rt = []
for i in response_rt['candles']:
    if i[0] not in date_temp:
        final_candles_list_rt.append(i)
        date_temp.append(i[0])
    else:
        print("duplicate detected ")
        
del date_temp
candles = np.array(response_rt['candles'])
candles.shape

In [ ]:
for i in candles[:, 0]:
    if float(i) in sync.data()[:, 0].tolist():
        print(datetime.datetime.fromtimestamp(int(i)).strftime("%D %H:%M:%S"))

In [ ]:
candles = np.array(response_rt['candles'])
candles

In [ ]:

datetime.datetime.fromtimestamp(int(sync.last_candle()[0])).strftime("%D %H %m")

In [ ]:
response_rt = fyers.history(data={
                                "symbol": "NSE:ZEEL-EQ",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_today,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })

date_temp = []
final_candles_list_rt = []
for i in response_rt['candles']:
    if i[0] not in date_temp:
        final_candles_list_rt.append(i)
        date_temp.append(i[0])
    else:
        print("duplicate detected ")
        
del date_temp
candles = np.array(response_rt['candles'])
candles.shape
for i in candles[:, 0]:
    print(datetime.datetime.fromtimestamp(int(i)).strftime("%D %H:%M:%S"))